<a href="https://colab.research.google.com/github/IBREEZZ/Generative_Ai_Project/blob/main/%D8%B1%D8%AD%D8%A7%D9%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The Final Project**

In [1]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.1 MB/s eta 0:00:00


In [2]:
import gradio as gr
import torch
from transformers import MarianTokenizer, MarianMTModel, pipeline
from sentence_transformers import SentenceTransformer, util
import re
from PyPDF2 import PdfReader
import requests
from datetime import datetime
from diffusers import StableDiffusionPipeline
from PIL import Image
import os
import tempfile

In [3]:
# ------ Constants and API Keys ------
OPENWEATHER_API_KEY = "e4123cf52bc39cf028fd5007f25bd3fd"

In [4]:
# ------ Model Initialization ------
print("🔄 Loading models...")

🔄 Loading models...


In [5]:
# Translation models
tokenizer_ar_en = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")
model_ar_en = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-ar-en")
tokenizer_en_ar = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ar")
model_en_ar = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-ar")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

In [6]:
# Embedding model
embedder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# Summarization pipeline (for Arabic)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [8]:
# ------ Core Functions ------
class OmanTourismAssistant:
    def __init__(self):
        self.chunks = []
        self.chunk_embeddings = torch.tensor([])
        self.generated_images = []
        self.image_pipe = None

        # Initialize image generation pipeline
        self._init_image_generator()

    def _init_image_generator(self):
        """Initialize the image generation pipeline"""
        try:
            self.image_pipe = StableDiffusionPipeline.from_pretrained(
                "runwayml/stable-diffusion-v1-5",
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                safety_checker=None,
                requires_safety_checker=False
            )

            if torch.cuda.is_available():
                self.image_pipe = self.image_pipe.to("cuda")
                print("✅ Image generator loaded on GPU")
            else:
                print("✅ Image generator loaded on CPU")

            self.image_pipe.enable_attention_slicing()
        except Exception as e:
            print(f"❌ Failed to load image generator: {str(e)}")

    def load_documents(self, files):
        """Process uploaded PDF/TXT files"""
        full_text = ""
        for file in files:
            try:
                if file.name.endswith(".pdf"):
                    reader = PdfReader(file)
                    text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
                elif file.name.endswith(".txt"):
                    with open(file.name, 'r', encoding='utf-8') as f:
                        text = f.read()
                full_text += text + "\n\n"
            except Exception as e:
                print(f"Error processing {file.name}: {str(e)}")

        self.chunks = self._split_text(full_text)
        if self.chunks:
            self.chunk_embeddings = embedder.encode(self.chunks, convert_to_tensor=True)
        return f"✅ تم تحميل {len(self.chunks)} جزء معلومات من {len(files)} ملف(ات)"

    def _split_text(self, text, max_length=300):
        """Split text into meaningful chunks"""
        sentences = re.split(r'(?<=[.!?؟])\s+', text)
        chunks = []
        current_chunk = []
        current_length = 0

        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue

            sent_length = len(sentence.split())
            if current_length + sent_length <= max_length:
                current_chunk.append(sentence)
                current_length += sent_length
            else:
                chunks.append(" ".join(current_chunk))
                current_chunk = [sentence]
                current_length = sent_length

        if current_chunk:
            chunks.append(" ".join(current_chunk))
        return chunks

    def _summarize_text(self, text, language="ar"):
        """Generate concise summary of text"""
        try:
            if language == "ar":
                summary = summarizer(text, max_length=130, min_length=30, do_sample=False)
                return summary[0]['summary_text']
            else:
                translated = self._translate(text[:500], "en")
                return "Summary: " + translated[:200] + "..."
        except:
            return text[:200] + "..." if len(text) > 200 else text

    def _get_weather(self, location):
        """Get weather information for a location"""
        try:
            url = f"http://api.openweathermap.org/data/2.5/weather?q={location},OM&appid={OPENWEATHER_API_KEY}&units=metric&lang=ar"
            response = requests.get(url).json()

            if response.get("cod") != 200:
                return None

            return {
                "description": response["weather"][0]["description"],
                "temp": response["main"]["temp"],
                "humidity": response["main"]["humidity"],
                "wind": response["wind"]["speed"]
            }
        except:
            return None

    def _translate(self, text, target_lang):
        """Translate text between Arabic and English"""
        if not text.strip():
            return text

        try:
            if target_lang == "en":
                tokenizer, model = tokenizer_ar_en, model_ar_en
            else:
                tokenizer, model = tokenizer_en_ar, model_en_ar

            inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
            outputs = model.generate(**inputs, max_length=512)
            return tokenizer.decode(outputs[0], skip_special_tokens=True)
        except:
            return text

    def _extract_location(self, text):
        """Extract location name from text"""
        locations = {
            "ar": ["صلالة", "نزوى", "مسقط", "صور", "طاقة", "مرباط", "وادي دربات", "عين رزات"],
            "en": ["salalah", "nizwa", "muscat", "sur", "taqa", "mirbat", "wadi darbat", "ain razat"]
        }

        for lang in locations:
            for loc in locations[lang]:
                if loc.lower() in text.lower():
                    return loc
        return "صلالة"

    def _generate_place_image(self, location, language):
        """Generate image for a tourist location"""
        if not self.image_pipe:
            return None

        try:
            # Create prompt based on language
            if language == "ar":
                prompt = f"منظر سياحي جميل لـ {location} في عمان، تصوير فوتوغرافي احترافي، دقة عالية"
                negative = "نص, كتابة, رسم, صورة غير واضحة, مشوهة, أشخاص"
            else:
                prompt = f"Beautiful tourist view of {location} in Oman, professional photography, 4k resolution"
                negative = "text, writing, drawing, blurry, distorted, people"

            # Generate image
            image = self.image_pipe(
                prompt=prompt,
                negative_prompt=negative,
                width=512,
                height=512,
                num_inference_steps=50,
                guidance_scale=7.5
            ).images[0]

            # Save image to temporary file
            temp_dir = tempfile.mkdtemp()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            image_path = os.path.join(temp_dir, f"generated_{location}_{timestamp}.jpg")
            image.save(image_path)

            self.generated_images.append(image_path)
            return image_path
        except Exception as e:
            print(f"Image generation error: {str(e)}")
            return None

    def _retrieve_information(self, query):
        """Retrieve relevant information chunks"""
        if not self.chunks:
            return []

        query_embedding = embedder.encode(query, convert_to_tensor=True)
        cos_scores = util.cos_sim(query_embedding, self.chunk_embeddings)[0]
        top_results = torch.topk(cos_scores, k=min(3, len(self.chunks)))

        return [{"text": self.chunks[idx], "score": score.item()}
                for score, idx in zip(top_results.values, top_results.indices)
                if score > 0.3]

    def generate_response(self, message, chat_history, language, generate_image=False):
        """Generate comprehensive response to user query"""
        response = ""
        image_path = None

        # Weather query
        weather_keywords = ["طقس", "جو", "حرارة", "weather", "temperature"]
        if any(kw in message.lower() for kw in weather_keywords):
            location = self._extract_location(message)
            weather = self._get_weather(location)

            if weather:
                if language == "ar":
                    response = (f"🌤️ الطقس في {location}:\n"
                               f"الحالة: {weather['description']}\n"
                               f"درجة الحرارة: {weather['temp']}°C\n"
                               f"الرطوبة: {weather['humidity']}%\n"
                               f"الرياح: {weather['wind']} م/ث")
                else:
                    response = (f"🌤️ Weather in {location}:\n"
                               f"Condition: {weather['description']}\n"
                               f"Temperature: {weather['temp']}°C\n"
                               f"Humidity: {weather['humidity']}%\n"
                               f"Wind: {weather['wind']} m/s")
            else:
                response = "⚠️ لا توجد بيانات طقس متاحة" if language == "ar" else "⚠️ No weather data available"

        # Information query
        else:
            chunks = self._retrieve_information(message)

            if chunks:
                combined = "\n\n".join([chunk['text'] for chunk in chunks[:2]])

                if language == "ar":
                    response = f"📌 المعلومات:\n{self._summarize_text(combined, 'ar')}"
                else:
                    translated = self._translate(combined, "en")
                    response = f"📌 Information:\n{self._summarize_text(translated, 'en')}"

                # Check if place query
                place_keywords = ["مكان", "معلم", "منطقة", "وادي", "عين", "شاطئ", "جبل",
                                "place", "attraction", "location", "valley", "spring", "beach", "mountain"]
                if any(kw in message.lower() for kw in place_keywords) and generate_image:
                    location = self._extract_location(message)
                    image_path = self._generate_place_image(location, language)
            else:
                response = "⚠️ لم أجد معلومات ذات صلة" if language == "ar" else "⚠️ No relevant information found"

        # Prepare the response with image if available
        if image_path:
            # Display both text and image in chat
            chat_history.append((message, [response, (image_path,)]))
        else:
            chat_history.append((message, response))

        return "", chat_history, image_path if image_path else None

In [9]:
# ------ Gradio Interface ------
assistant = OmanTourismAssistant()

with gr.Blocks(title="رحال عُمان السياحي", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    <div style="text-align: center;">
        <h1>رحال عُمان السياحي الذكي</h1>
        <p>اسأل عن أي مكان سياحي في عُمان واحصل على معلومات دقيقة مع صور توضيحية</p>
    </div>
    """)

    with gr.Tab("تحميل الملفات"):
        files = gr.File(label="ارفع ملفات PDF أو TXT", file_types=[".pdf", ".txt"], file_count="multiple")
        upload_btn = gr.Button("معالجة الملفات")
        upload_status = gr.Textbox(label="حالة التحميل", interactive=False)

    with gr.Tab("الدردشة"):
        lang = gr.Radio(["ar", "en"], label="اللغة", value="ar")
        chatbot = gr.Chatbot(height=500)
        msg = gr.Textbox(label="اكتب سؤالك هنا")

        with gr.Row():
            send_btn = gr.Button("إرسال")
            img_btn = gr.Button("إرسال مع صورة")
            clear_btn = gr.Button("مسح المحادثة")

    with gr.Tab("الصور المولدة"):
        gallery = gr.Gallery(label="معرض الصور", columns=3, height=600)

    # Examples
    gr.Examples(
        examples=[
            ["what is the weather in Muscat?", "en"],
            ["أخبرني عن الطقس في صلالة", "ar"],
            ["Tell me about Nizwa Fort", "en"],
            ["اخبرني عن قلعة بهلاء", "an"]
        ],
        inputs=[msg, lang],
        label="أمثلة استعلامات"
    )

    # Event handlers
    upload_btn.click(
        assistant.load_documents,
        inputs=[files],
        outputs=[upload_status]
    )

    def respond(message, chat_history, language, generate_image):
        response, new_chat_history, image_path = assistant.generate_response(
            message, chat_history, language, generate_image
        )
        return response, new_chat_history, image_path if image_path else None

    send_btn.click(
        respond,
        inputs=[msg, chatbot, lang, gr.State(False)],
        outputs=[msg, chatbot, gallery]
    )

    img_btn.click(
        respond,
        inputs=[msg, chatbot, lang, gr.State(True)],
        outputs=[msg, chatbot, gallery]
    )

    msg.submit(
        respond,
        inputs=[msg, chatbot, lang, gr.State(False)],
        outputs=[msg, chatbot, gallery]
    )

    clear_btn.click(lambda: None, None, chatbot, queue=False)

# Launch the app
if __name__ == "__main__":
    demo.launch(share=True)

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

✅ Image generator loaded on GPU


/tmp/ipython-input-9-721771238.py:19: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cf285e3c54f3b09670.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
